In [1]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor




In [2]:
cancer = pd.read_csv("BreastCancer.csv",index_col = 0)

In [3]:
cancer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 699 entries, 61634 to 13454352
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Clump          699 non-null    int64 
 1   UniCell_Size   699 non-null    int64 
 2   Uni_CellShape  699 non-null    int64 
 3   MargAdh        699 non-null    int64 
 4   SEpith         699 non-null    int64 
 5   BareN          699 non-null    int64 
 6   BChromatin     699 non-null    int64 
 7   NoemN          699 non-null    int64 
 8   Mitoses        699 non-null    int64 
 9   Class          699 non-null    object
dtypes: int64(9), object(1)
memory usage: 60.1+ KB


In [4]:
X = cancer.drop('Class', axis = 1)
y = cancer['Class']

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify= y)

In [6]:
dtc = DecisionTreeClassifier(random_state= 24,max_depth = 2)
lr = LogisticRegression(random_state= 24, C = 0.4)
nb = GaussianNB()

In [7]:
voting = VotingClassifier([('DT',dtc),('LR',lr),('NB',nb)],voting = 'soft') # default voting = hard

In [8]:
voting.fit(X_train,y_train)


VotingClassifier(estimators=[('DT',
                              DecisionTreeClassifier(max_depth=2,
                                                     random_state=24)),
                             ('LR', LogisticRegression(C=0.4, random_state=24)),
                             ('NB', GaussianNB())],
                 voting='soft')

In [9]:
y_pred =  voting.predict(X_test)

In [10]:
accuracy = accuracy_score(y_test,y_pred)

In [11]:
accuracy

0.9761904761904762

In [12]:
y_pred_prob = voting.predict_proba(X_test)[:,1]

In [13]:
y_pred_prob

array([0.0100393 , 0.01568926, 0.99040296, 0.98931768, 0.01227428,
       0.02727753, 0.01400488, 0.01227428, 0.01244184, 0.0100393 ,
       0.9088534 , 0.98970868, 0.01031831, 0.01683726, 0.99173922,
       0.01442019, 0.01072691, 0.98963949, 0.96858489, 0.99170619,
       0.01952104, 0.99137428, 0.01622395, 0.88586366, 0.99097407,
       0.67983598, 0.98413084, 0.9086694 , 0.01050189, 0.01031831,
       0.01059432, 0.01094967, 0.02647168, 0.01280417, 0.02132397,
       0.25331137, 0.01154419, 0.01054287, 0.99179835, 0.0100393 ,
       0.61854623, 0.01061186, 0.99088113, 0.99145271, 0.90377568,
       0.74521895, 0.91629103, 0.99177074, 0.01137614, 0.98910144,
       0.99178138, 0.98886498, 0.0100393 , 0.01054287, 0.9072914 ,
       0.01056784, 0.67946023, 0.0100393 , 0.01133499, 0.01154419,
       0.01214378, 0.83190116, 0.01063867, 0.01043035, 0.99177299,
       0.99177146, 0.9796035 , 0.0106786 , 0.01058574, 0.0129329 ,
       0.91290672, 0.98339952, 0.02293749, 0.01175193, 0.02272

In [14]:
score = roc_auc_score(y_test,y_pred_prob)

In [15]:
score

0.9938607085346216

##GridSearchCV

In [17]:
dtc = DecisionTreeClassifier(random_state= 24,max_depth = 2)
lr = LogisticRegression(random_state= 24, C = 0.4)
nb = GaussianNB()
voting = VotingClassifier([('DT',dtc),('LR',lr),('NB',nb)],voting = 'soft') # default voting = hard

In [18]:
params = {'DT__min_samples_split':[2,10,20],
          'DT__min_samples_leaf':[1,10,20],
          'DT__max_depth':[None,2,3],
          'LR__C': np.linspace(0.001,3,5)
         }
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 24)
gcv = GridSearchCV(voting,param_grid = params,scoring = 'roc_auc',
                   cv = kfold, verbose = 3)

In [19]:
gcv.fit(X,y)

Fitting 5 folds for each of 135 candidates, totalling 675 fits
[CV 1/5] END DT__max_depth=None, DT__min_samples_leaf=1, DT__min_samples_split=2, LR__C=0.001;, score=0.997 total time=   0.0s
[CV 2/5] END DT__max_depth=None, DT__min_samples_leaf=1, DT__min_samples_split=2, LR__C=0.001;, score=0.995 total time=   0.0s
[CV 3/5] END DT__max_depth=None, DT__min_samples_leaf=1, DT__min_samples_split=2, LR__C=0.001;, score=0.994 total time=   0.0s
[CV 4/5] END DT__max_depth=None, DT__min_samples_leaf=1, DT__min_samples_split=2, LR__C=0.001;, score=0.990 total time=   0.0s
[CV 5/5] END DT__max_depth=None, DT__min_samples_leaf=1, DT__min_samples_split=2, LR__C=0.001;, score=0.993 total time=   0.0s
[CV 1/5] END DT__max_depth=None, DT__min_samples_leaf=1, DT__min_samples_split=2, LR__C=0.75075;, score=0.995 total time=   0.0s
[CV 2/5] END DT__max_depth=None, DT__min_samples_leaf=1, DT__min_samples_split=2, LR__C=0.75075;, score=0.995 total time=   0.0s
[CV 3/5] END DT__max_depth=None, DT__min_sam

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=VotingClassifier(estimators=[('DT',
                                                     DecisionTreeClassifier(max_depth=2,
                                                                            random_state=24)),
                                                    ('LR',
                                                     LogisticRegression(C=0.4,
                                                                        random_state=24)),
                                                    ('NB', GaussianNB())],
                                        voting='soft'),
             param_grid={'DT__max_depth': [None, 2, 3],
                         'DT__min_samples_leaf': [1, 10, 20],
                         'DT__min_samples_split': [2, 10, 20],
                         'LR__C': array([1.00000e-03, 7.50750e-01, 1.50050e+00, 2.25025e+00, 3.00000e+00])},
             scoring='roc_auc', verbose=3)

In [20]:
print(gcv.best_params_)
print(gcv.best_score_)

{'DT__max_depth': None, 'DT__min_samples_leaf': 1, 'DT__min_samples_split': 2, 'LR__C': 0.001}
0.9937986416496842


In [21]:
model = gcv.best_estimator_

In [22]:
model

VotingClassifier(estimators=[('DT', DecisionTreeClassifier(random_state=24)),
                             ('LR',
                              LogisticRegression(C=0.001, random_state=24)),
                             ('NB', GaussianNB())],
                 voting='soft')

In [23]:
model.fit(X_train,y_train)

VotingClassifier(estimators=[('DT', DecisionTreeClassifier(random_state=24)),
                             ('LR',
                              LogisticRegression(C=0.001, random_state=24)),
                             ('NB', GaussianNB())],
                 voting='soft')

In [26]:
y_pred_prob = model.predict_proba(X_test)[:,1]

In [28]:
roc_auc_score(y_test, y_pred_prob)

0.9942632850241546